In [1]:
import numpy as np
from pyscf import gto, scf, ao2mo, fci

script that contains collection of approximate 2RDMs from 1RDMFT literature

The following function caculates the electron electron interactio  energy from the Fock basis representation of the 2RDM $\gamma_2$ in the follwong way:
$$E_{ee} = \sum_p \sum_q \sum_r \sum_s \gamma_{2_{pqrs}} \langle p q | r s \rangle$$

In [2]:
def ee_from_gamma_2(eri,dm2):
    return .5*np.sum(np.multiply(eri,dm2))

The matrix form of $\gamma_2$ in the Hartree Fock approximation in the basis $\{\eta\}$ for spin restricted natural orbitals, i.e. $\{\eta_{\alpha}\} =\{\eta_{\beta}\} $  is:

$$
\begin{split}
    \gamma_{2_{ijkl}} &=  n_i n_k 4  \delta_{ij} \delta_{kl} - n_i n_k 2 \delta_{il} \delta_{jk}
\end{split}
$$
  
where $n_i \in [0,1]$, and this is a implementation:



In [3]:
def HF_2RDM(n, M):
    ''' Compute 2 RDMFTs in Natural Orbital basis for Hartree Fock approximation
    Parameters
    ----------
    n : np.ndarray
        occupation numbers of a spin restricted 1dm, they lie in [0,2]
    M : integer 
        basis set size

    Returns
    -------
    rho : np.ndarray
        2RDM
    '''
    # this form is for spin restriced orbitals 
    TWORDM = np.zeros((M,M,M,M))
    for i in range(0,M):
        for j in range(0,M):
            for k in range(0,M):
                for l in range(0,M):
                    if i==j and k==l:
                        TWORDM[i,j,k,l] = 4
                    if i==l and k==j:
                        TWORDM[i,j,k,l] -= 2
                    TWORDM[i,j,k,l] = n[i]/2*n[k]/2*TWORDM[i,j,k,l]

    return TWORDM

The matrix form of $\gamma_2$ in the Mueller approximation in the basis $\{\eta\}$ for spin restricted natural orbitals, i.e. $\{\eta_{\alpha}\} =\{\eta_{\beta}\} $  is:

$$
\begin{split}
    \gamma_{2_{ijkl}} &=  n_i n_k 4  \delta_{ij} \delta_{kl} - \sqrt{n_i n_k} 2 \delta_{il} \delta_{jk}
\end{split}
$$
  
where $n_i \in [0,1]$, and this is a implementation:

In [4]:
def MU_2RDM(n, M):
    ''' Compute 2 RDMFTs in Natural Orbital basis for Mueller approximation
    Parameters
    ----------
    n : np.ndarray
        occupation numbers of a spin restricted 1dm, they lie in [0,2]
    M : integer 
        basis set size

    Returns
    -------
    rho : np.ndarray
        2RDM
    '''
    TWORDM = np.zeros((M,M,M,M))
    for i in range(0,M):
        for j in range(0,M):
            for k in range(0,M):
                for l in range(0,M):
                    if i==j and k==l:
                        TWORDM[i,j,k,l] = 4*n[i]/2*n[k]/2
                    if i==l and k==j:
                        TWORDM[i,j,k,l] -= 2*np.sqrt(n[i]/2*n[k]/2)
    return TWORDM

The matrix form of $\gamma_2$ in the Mueller approximation in the basis $\{\eta\}$ for spin restricted natural orbitals, i.e. $\{\eta_{\alpha}\} =\{\eta_{\beta}\} $  is:

$$
\begin{split}
    \gamma_{2_{ijkl}} &=  n_i n_k 4  \delta_{ij} \delta_{kl} - (n_i n_k)^{\alpha} 2 \delta_{il} \delta_{jk}
\end{split}
$$
  
where $n_i \in [0,1]$, and this is a implementation:

In [5]:
def POW_2RDM(n, M, alpha):
    ''' Compute 2 RDMFTs in Natural Orbital basis for Power Functional approximation
    Parameters
    ----------
    n : np.ndarray
        occupation numbers of a spin restricted 1dm, they lie in [0,2]
    M : integer 
        basis set size
    alpha: float
         power of the power functional

    Returns
    -------
    rho : np.ndarray
        2RDM
    '''
    TWORDM = np.zeros((M,M,M,M))
    for i in range(0,M):
        for j in range(0,M):
            for k in range(0,M):
                for l in range(0,M):
                    if i==j and k==l:
                        TWORDM[i,j,k,l] = 4*n[i]/2*n[k]/2
                    if i==l and k==j:
                        TWORDM[i,j,k,l] -= 2*(n[i]/2*n[k]/2)**(alpha)
    return TWORDM

The matrix form of $\gamma_2$ in the Goedecker-Umrigar approximation in the basis $\{\eta\}$ for spin restricted natural orbitals, i.e. $\{\eta_{\alpha}\} =\{\eta_{\beta}\} $  is:

$$
\begin{split}
    \gamma_{2_{ijkl}} &=  n_i n_k 2  \delta_{ij} \delta_{kl} (1-\delta_{jk}) +  n_i n_k 2  \delta_{ij} \delta_{kl} - \sqrt{n_i n_k} 2 \delta_{il} \delta_{jk} (1-\delta_{lk})
\end{split}
$$
  
where $n_i \in [0,1]$, and this is a implementation:

In [6]:
def GU_2RDM(n, M):
    ''' Compute 2 RDMFTs in Natural Orbital basis for Goedecker Umrigar approximation
    Parameters
    ----------
    n : np.ndarray
        occupation numbers of a spin restricted 1dm, they lie in [0,2]
    M : integer 
        basis set size
    Returns
    -------
    rho : np.ndarray
        2RDM
    '''    
    TWORDM = np.zeros((M,M,M,M))
    for i in range(0,M):
        for j in range(0,M):
            for k in range(0,M):
                for l in range(0,M):
                    if i==j and k==l and j!=k:
                        TWORDM[i,j,k,l] = 4*n[i]/2*n[k]/2
                    if i==l and k==j and l!=k :
                        TWORDM[i,j,k,l] -= 2*np.sqrt(n[i]/2*n[k]/2)
                    if i==j and k==l and j==k:
                         TWORDM[i,j,k,l] = 2*n[i]/2*n[k]/2


    return TWORDM

The matrix form of $\gamma_2$ in the Bujiese Baenrends Correction 1 approximation in the basis $\{\eta\}$ for spin restricted natural orbitals, i.e. $\{\eta_{\alpha}\} =\{\eta_{\beta}\} $  is:

$$
\begin{split}
    \gamma_{2_{ijkl}} &= 4 n_i n_k   \delta_{ij} \delta_{kl} - 2 \sqrt{n_i n_k}  \delta_{il} \delta_{jk} + 4 \sqrt{n_i n_k}  \delta_{il} \delta_{jk} (1-\delta_{ij}) \Theta(i - N)  \Theta(k - N)
\end{split}
$$
  
where $n_i \in [0,1]$, and this is a implementation:

In [7]:
def BBC1_2RDM(n, M, N):
    ''' Compute 2 RDMFTs in Natural Orbital basis for Goedecker Umrigar approximation
    Parameters
    ----------
    n : np.ndarray
        occupation numbers of a spin restricted 1dm, they lie in [0,2]
    M : integer 
        basis set size
    N : integer 
        number of strongly occupied (close to 1) orbitals
    Returns
    -------
    rho : np.ndarray
        2RDM
    '''    
    TWORDM = np.zeros((M,M,M,M))
    for i in range(0,M):
        for j in range(0,M):
            for k in range(0,M):
                for l in range(0,M):
                    if i==j and k==l:
                        TWORDM[i,j,k,l] = 4*n[i]/2*n[k]/2
                    if i==l and k==j :
                        TWORDM[i,j,k,l] -= 2*np.sqrt(n[i]/2*n[k]/2)
                    if i==l and k==j and (i >= N and k >= N) and l!=k :
                        TWORDM[i,j,k,l] += 4*np.sqrt(n[i]/2*n[k]/2)
                    
    return TWORDM

In [8]:
mol = gto.Mole()
mol.atom = """
    Be    0.    0.    0.
"""
mol.basis = "cc-pvdz"
#mol.basis = "6-31g" 
mol.build()

In [9]:
# Run Hartree-Fock.
mf = scf.RHF(mol)
mf.kernel()

converged SCF energy = -14.5723376309534


np.float64(-14.572337630953367)

In [10]:
# Find electron-repulsion integrals (eri).
eri = ao2mo.kernel(mol, mf.mo_coeff)
h2_MO = np.asarray(ao2mo.restore(1, eri, mol.nao))

In [11]:
# First create FCI solver with function fci.FCI and solve the FCI problem
cisolver = fci.FCI(mol, mf.mo_coeff)
e, fcivec = cisolver.kernel()

In [12]:
# generate \gamma_1, \gamma_2, the natural orbitals and occupation numbers of FCI
dm1, fci_dm2 = cisolver.make_rdm12(fcivec, mf.mo_coeff.shape[1], mol.nelec)
FCIoccu, FCInaturalC = np.linalg.eigh(dm1)

In [13]:
# sort, so that weakly occupied orbitals are first N entries
FCInaturalC =  FCInaturalC[:,::-1]
FCIoccu = FCIoccu[::-1]

In [14]:
# convert the elements of h2 and the elements of \gamma_2 from the MO basis to the NO basis 
eri_fciNAO = h2_MO.copy()
fci_dm2_fciNAO = fci_dm2.copy()
for i in range(4):
  eri_fciNAO = np.tensordot(eri_fciNAO, FCInaturalC, axes=1).transpose(3, 0, 1, 2)
  fci_dm2_fciNAO = np.tensordot(fci_dm2_fciNAO, FCInaturalC, axes=1).transpose(3, 0, 1, 2)



In [15]:
hf_dm2 = HF_2RDM(FCIoccu, h2_MO.shape[0])
mu_dm2 = MU_2RDM(FCIoccu, h2_MO.shape[0])
gu_dm2 = GU_2RDM(FCIoccu, h2_MO.shape[0])
bbc1_dm2 = BBC1_2RDM(FCIoccu, h2_MO.shape[0], mol.nelec[0])

In [16]:
print(f"{'FCI:':10s}", ee_from_gamma_2(eri_fciNAO,fci_dm2_fciNAO))
print(f"{'HF:':10s}", ee_from_gamma_2(eri_fciNAO,hf_dm2))
print(f"{'Mueller:':10s}", ee_from_gamma_2(eri_fciNAO,mu_dm2))
print(f"{'Goedecker:':10s}", ee_from_gamma_2(eri_fciNAO,gu_dm2))
print(f"{'BBC1:':10s}", ee_from_gamma_2(eri_fciNAO,bbc1_dm2))

FCI:       4.460572797091583
HF:        4.582517653301028
Mueller:   4.4296509155290344
Goedecker: 4.493532910096157
BBC1:      4.442846537142929
